In [ ]:
import os
from boxsdk import OAuth2, Client
from boxsdk.exception import BoxAPIException
import requests
import json


# OAuth 2.0 credentials
client_id = 'vc9saqpuvmiimz3agdvvdcrzzqudrrty'
client_secret = 'eQHxqemYksHP5lrsrhTlntTHFZCGMy5N'
access_token = 'TPQ5AfgsIi2DWaFLk5eLxItowjIvd06j'

# Create OAuth2 instance
oauth = OAuth2(
    client_id=client_id,
    client_secret=client_secret,
    access_token=access_token
)

# Create a Box client
client = Client(oauth)

def upload_file(file_path, parent_folder_id):
    file_name = os.path.basename(file_path)
    client.folder(parent_folder_id).upload(file_path, file_name)

# Function to upload a folder and its contents recursively
def upload_folder(folder_path, parent_folder_id):
    folder_name = os.path.basename(folder_path)

    # Create a new folder inside the parent folder
    new_folder = client.folder(parent_folder_id).create_subfolder(folder_name)
    new_folder_id = new_folder.id

    # Iterate over the files and subfolders in the folder
    for item_name in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item_name)

        if os.path.isfile(item_path):
            # Upload the file
            upload_file(item_path, new_folder_id)
        elif os.path.isdir(item_path):
            # Recursively upload the subfolder
            upload_folder(item_path, new_folder_id)

# Example usage: Upload each folder and its contents to "Beamline Sim Data" folder
root_folder = 'E://data/'
parent_folder_id = '215071014756'

# Iterate over the subfolders in the root folder
for folder_name in os.listdir(root_folder):
    folder_path = os.path.join(root_folder, folder_name)
    
    if os.path.isdir(folder_path):
        upload_folder(folder_path, parent_folder_id)


In [5]:
from boxsdk import OAuth2, Client

# OAuth 2.0 credentials
client_id = 'vc9saqpuvmiimz3agdvvdcrzzqudrrty'
client_secret = 'eQHxqemYksHP5lrsrhTlntTHFZCGMy5N'
access_token = '4Ec5xTNCmgMadVUb05I4x0z9jUaGzVuG'

# Create OAuth2 instance
oauth = OAuth2(
    client_id=client_id,
    client_secret=client_secret,
    access_token=access_token
)

# Create a Box client
client = Client(oauth)

# Function to get all folder names and IDs recursively
def get_all_folder_info(parent_folder_id):
    folder_info = []

    # Get the folder items in the parent folder
    items = client.folder(folder_id=parent_folder_id).get_items()

    for item in items:
        if item.type == 'folder':
            # Append the folder name and ID to the list
            folder_info.append({
                'name': item.name,
                'id': item.id
            })

            # Recursively get folder info in the subfolder
            subfolder_info = get_all_folder_info(item.id)
            folder_info.extend(subfolder_info)

    return folder_info

# Example usage: Get all folder names and IDs in the Box account
root_folder_id = '0'

# Get all folder names and IDs recursively starting from the root folder
all_folder_info = get_all_folder_info(root_folder_id)

# Print the folder names and IDs
for folder in all_folder_info:
    print(f"Folder Name: {folder['name']}, Folder ID: {folder['id']}")


Folder Name: Box Tips, Folder ID: 1146422989
Folder Name: Communications Toolkit, Folder ID: 3895184399
Folder Name: Additional Guidelines, Folder ID: 5699484997
Folder Name: Argonne Public Website Guidelines and References, Folder ID: 51272495974
Folder Name: Drupal 8 Content Templates for new site, Folder ID: 52532659267
Folder Name: Sponsor Guidelines, Folder ID: 127711940675
Folder Name: Argonne Social Medial Profile Banners, Folder ID: 161660260969
Folder Name: LinkedIn Profile Banners, Folder ID: 161660671453
Folder Name: Argonne Virtual Meeting Backgrounds, Folder ID: 116253036058
Folder Name: ANL_75th Anniversary_Meeting Backgrounds, Folder ID: 149866079108
Folder Name: ANL_AET Backgrounds, Folder ID: 161213680386
Folder Name: ANL_ALCF Backgrounds, Folder ID: 149868156158
Folder Name: ANL_APS-U Backgrounds, Folder ID: 149867092535
Folder Name: ANL_Argonne in Chicago Backgrounds, Folder ID: 149866629429
Folder Name: ANL_CNM Backgrounds, Folder ID: 149867520963
Folder Name: ANL_E